In [13]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv("./movie_ratings.csv")

In [15]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [16]:
df.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [17]:
# filter to likes (rating ≥ 4)
df = df[df.rating >= 4]

In [18]:
df.head()

,userId,movieId,rating,timestamp
4,1,1172,4.0,1260759205
12,1,1953,4.0,1260759191
13,1,2105,4.0,1260759139
20,2,10,4.0,835355493
21,2,17,5.0,835355681


In [19]:
df.userId.value_counts()

564    1115
547    1031
30      702
73      617
472     544
       ... 
170       2
429       2
579       1
79        1
221       1
Name: userId, Length: 671, dtype: int64

In [20]:
# get max_rating for user
df = df.join(df.groupby('userId')['rating'].max().rename('user_max_rating'), on='userId')
df

,userId,movieId,rating,timestamp,user_max_rating
4,1,1172,4.0,1260759205,4.0
12,1,1953,4.0,1260759191,4.0
13,1,2105,4.0,1260759139,4.0
20,2,10,4.0,835355493,5.0
21,2,17,5.0,835355681,5.0
...,...,...,...,...,...
99995,671,5989,4.0,1064890625,5.0
99996,671,5991,4.5,1064245387,5.0
99997,671,5995,4.0,1066793014,5.0
100000,671,6269,4.0,1065149201,5.0


In [21]:
# for each user get min movieId with highest rating => testMovies
testMovies = df[df['rating'] == df['user_max_rating']].groupby('userId')['movieId'].min().rename('test_movie')
# mark test movies
df = df.join(testMovies, on='userId')
df['is_test'] = df['movieId'] == df['test_movie']
# drop tmp columns
df.drop(['user_max_rating', 'test_movie'], axis=1, inplace=True)

df

,userId,movieId,rating,timestamp,is_test
4,1,1172,4.0,1260759205,True
12,1,1953,4.0,1260759191,False
13,1,2105,4.0,1260759139,False
20,2,10,4.0,835355493,False
21,2,17,5.0,835355681,True
...,...,...,...,...,...
99995,671,5989,4.0,1064890625,False
99996,671,5991,4.5,1064245387,False
99997,671,5995,4.0,1066793014,False
100000,671,6269,4.0,1065149201,False


In [22]:
train = df[df['is_test'] == False]
train

,userId,movieId,rating,timestamp,is_test
12,1,1953,4.0,1260759191,False
13,1,2105,4.0,1260759139,False
20,2,10,4.0,835355493,False
22,2,39,5.0,835355604,False
23,2,47,4.0,835355552,False
...,...,...,...,...,...
99995,671,5989,4.0,1064890625,False
99996,671,5991,4.5,1064245387,False
99997,671,5995,4.0,1066793014,False
100000,671,6269,4.0,1065149201,False


In [23]:
test = df[df['is_test']]
test

,userId,movieId,rating,timestamp,is_test
4,1,1172,4.0,1260759205,True
21,2,17,5.0,835355681,True
101,3,318,5.0,1298862121,True
148,4,34,5.0,949919556,True
368,5,597,5.0,1163373711,True
...,...,...,...,...,...
99739,667,32,5.0,847271481,True
99801,668,296,5.0,993613478,True
99822,669,260,5.0,1015829081,True
99859,670,25,5.0,938782344,True


In [24]:
# write train/test datasets
train.to_csv("../data/movie_ratings/movie_ratings_train.csv")
train.to_csv("./movie_ratings_train.csv")
test.to_csv("./movie_ratings_test.csv")

print("done")

done
